In [29]:
import warnings
warnings.simplefilter('ignore')

import gc

import numpy as np
import pandas as pd
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)
from tqdm.notebook import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

import lightgbm as lgb

In [3]:
train = pd.read_csv('train.csv')
train = train.sort_values(by=['QUEUE_ID', 'DOTTING_TIME']).reset_index(drop=True)

test = pd.read_csv('evaluation_public.csv')
test = test.sort_values(by=['ID', 'DOTTING_TIME']).reset_index(drop=True)

In [8]:
train.head()

,QUEUE_ID,CU,QUEUE_TYPE,CPU_USAGE,MEM_USAGE,LAUNCHING_JOB_NUMS,RUNNING_JOB_NUMS,SUCCEED_JOB_NUMS,CANCELLED_JOB_NUMS,FAILED_JOB_NUMS,DOTTING_TIME,DISK_USAGE
0,2,16,2,3,54,0,0,0,0,0,1590683100000,20.0
1,2,16,2,2,54,0,0,0,0,0,1590683400000,20.0
2,2,16,2,7,54,0,0,0,0,0,1590683700000,20.0
3,2,16,2,4,54,0,0,0,0,0,1590684000000,20.0
4,2,16,2,5,54,0,0,0,0,0,1590684120000,20.0


In [9]:
test.head()

,ID,QUEUE_ID,CU,QUEUE_TYPE,CPU_USAGE,MEM_USAGE,LAUNCHING_JOB_NUMS,RUNNING_JOB_NUMS,SUCCEED_JOB_NUMS,CANCELLED_JOB_NUMS,FAILED_JOB_NUMS,DOTTING_TIME,DISK_USAGE
0,1,297,16,2,60,69,0,5,5,0,0,1662213420000,9
1,1,297,16,2,58,69,0,9,4,0,0,1662213720000,9
2,1,297,16,2,80,67,0,9,1,0,0,1662214020000,9
3,1,297,16,2,100,65,0,7,2,0,1,1662214320000,9
4,1,297,16,2,98,67,0,10,3,0,1,1662214620000,9


In [4]:
train.shape, test.shape

((501730, 15), (14980, 16))

In [5]:
# 这些 columns 在 test 只有单一值, 所以直接去掉

del train['STATUS']
del train['PLATFORM']
del train['RESOURCE_TYPE']

del test['STATUS']
del test['PLATFORM']
del test['RESOURCE_TYPE']

In [6]:
# Label Encoding

le = LabelEncoder()
train['QUEUE_TYPE'] = le.fit_transform(train['QUEUE_TYPE'].astype(str))
test['QUEUE_TYPE'] = le.transform(test['QUEUE_TYPE'].astype(str))

In [7]:
train.isnull().sum()

QUEUE_ID                  0
CU                        0
QUEUE_TYPE                0
CPU_USAGE                 0
MEM_USAGE                 0
LAUNCHING_JOB_NUMS        0
RUNNING_JOB_NUMS          0
SUCCEED_JOB_NUMS          0
CANCELLED_JOB_NUMS        0
FAILED_JOB_NUMS           0
DOTTING_TIME              0
DISK_USAGE            33095
dtype: int64

In [11]:
# 只用 CPU_USAGE 和 MEM_USAGE，CU 及 QUEUE_TYPE （提取）

keep_cols_train = ['QUEUE_ID', 'CU', 'QUEUE_TYPE', 'MEM_USAGE', 'CPU_USAGE']
keep_cols_test = ['ID', 'QUEUE_ID', 'CU', 'QUEUE_TYPE', 'MEM_USAGE', 'CPU_USAGE']

train = train[keep_cols_train]
test = test[keep_cols_test]

In [12]:
train.head()

,QUEUE_ID,CU,QUEUE_TYPE,MEM_USAGE,CPU_USAGE
0,2,16,2,54,3
1,2,16,2,54,2
2,2,16,2,54,7
3,2,16,2,54,4
4,2,16,2,54,5


In [13]:
test.head()

,ID,QUEUE_ID,CU,QUEUE_TYPE,MEM_USAGE,CPU_USAGE
0,1,297,16,2,69,60
1,1,297,16,2,69,58
2,1,297,16,2,67,80
3,1,297,16,2,65,100
4,1,297,16,2,67,98


In [16]:
merge_cols = ['MEM_USAGE', 'CPU_USAGE']

In [18]:
# 处理train

for i in range(1, 6):
    cols = [col + "_" + str(i) for col in merge_cols]
    train[cols] = train.groupby('QUEUE_ID')[merge_cols].shift(-1 * i + 1)
    
for i in range(1, 6):
    cpu_name = 'cpu_' + str(i)
    train[cpu_name] = train.groupby('QUEUE_ID')['CPU_USAGE'].shift(-1 * i - 4)
    
train.drop(columns=merge_cols, inplace=True)

In [19]:
train.head()

,QUEUE_ID,CU,QUEUE_TYPE,MEM_USAGE_1,CPU_USAGE_1,MEM_USAGE_2,CPU_USAGE_2,MEM_USAGE_3,CPU_USAGE_3,MEM_USAGE_4,CPU_USAGE_4,MEM_USAGE_5,CPU_USAGE_5,cpu_1,cpu_2,cpu_3,cpu_4,cpu_5
0,2,16,2,54,3,54.0,2.0,54.0,7.0,54.0,4.0,54.0,5.0,3.0,2.0,2.0,5.0,6.0
1,2,16,2,54,2,54.0,7.0,54.0,4.0,54.0,5.0,55.0,3.0,2.0,2.0,5.0,6.0,2.0
2,2,16,2,54,7,54.0,4.0,54.0,5.0,55.0,3.0,54.0,2.0,2.0,5.0,6.0,2.0,3.0
3,2,16,2,54,4,54.0,5.0,55.0,3.0,54.0,2.0,54.0,2.0,5.0,6.0,2.0,3.0,10.0
4,2,16,2,54,5,55.0,3.0,54.0,2.0,54.0,2.0,54.0,5.0,6.0,2.0,3.0,10.0,6.0


In [20]:
# 处理test

for i in range(1, 6):
    cols = [col + "_" + str(i) for col in merge_cols]
    test[cols] = test.groupby(['ID', 'QUEUE_ID'])[merge_cols].shift(-1 * i + 1)
    
test.drop(columns=merge_cols, inplace=True)

In [21]:
test.head()

,ID,QUEUE_ID,CU,QUEUE_TYPE,MEM_USAGE_1,CPU_USAGE_1,MEM_USAGE_2,CPU_USAGE_2,MEM_USAGE_3,CPU_USAGE_3,MEM_USAGE_4,CPU_USAGE_4,MEM_USAGE_5,CPU_USAGE_5
0,1,297,16,2,69,60,69.0,58.0,67.0,80.0,65.0,100.0,67.0,98.0
1,1,297,16,2,69,58,67.0,80.0,65.0,100.0,67.0,98.0,NaN,NaN
2,1,297,16,2,67,80,65.0,100.0,67.0,98.0,NaN,NaN,NaN,NaN
3,1,297,16,2,65,100,67.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1,297,16,2,67,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
train = train.dropna()
test = test.dropna()

In [25]:
train.shape, test.shape

((501343, 18), (2996, 14))

In [28]:
train['cpu_mean'] = train[[f'CPU_USAGE_{i}' for i in range(1,6)]].mean(axis=1)
train['cpu_std'] = train[[f'CPU_USAGE_{i}' for i in range(1,6)]].std(axis=1)
train['cpu_diff'] = train['CPU_USAGE_5'] - train['CPU_USAGE_1']
train['cpu_max'] = train[[f'CPU_USAGE_{i}' for i in range(1,6)]].max(axis=1)
train['mem_mean'] = train[[f'MEM_USAGE_{i}' for i in range(1,6)]].mean(axis=1)
train['mem_std'] = train[[f'MEM_USAGE_{i}' for i in range(1,6)]].std(axis=1)
train['mem_max'] = train[[f'MEM_USAGE_{i}' for i in range(1,6)]].max(axis=1)
train['mem_diff'] = train['MEM_USAGE_5'] - train['MEM_USAGE_1']

test['cpu_mean'] = test[[f'CPU_USAGE_{i}' for i in range(1,6)]].mean(axis=1)
test['cpu_std'] = test[[f'CPU_USAGE_{i}' for i in range(1,6)]].std(axis=1)
test['cpu_diff'] = test['CPU_USAGE_5'] - test['CPU_USAGE_1']
test['cpu_max'] = test[[f'CPU_USAGE_{i}' for i in range(1,6)]].max(axis=1)
test['mem_mean'] = test[[f'MEM_USAGE_{i}' for i in range(1,6)]].mean(axis=1)
test['mem_std'] = test[[f'MEM_USAGE_{i}' for i in range(1,6)]].std(axis=1)
test['mem_max'] = test[[f'MEM_USAGE_{i}' for i in range(1,6)]].max(axis=1)
test['mem_diff'] = test['MEM_USAGE_5'] - test['MEM_USAGE_1']

In [32]:
def run_lgb_qid(train, test, target, qid):
    
    # 提取特征列
    feature_names = list(
        filter(lambda x: x not in ['QUEUE_ID', 'CU', 'QUEUE_TYPE'] + [f'cpu_{i}' for i in range(1,6)], 
               train.columns))
    
    # 提取 QUEUE_ID 对应的数据集
    train = train[train.QUEUE_ID == qid]
    test = test[test.QUEUE_ID == qid]
    
    print(f"QUEUE_ID:{qid}, target:{target}, train:{len(train)}, test:{len(test)}")
    
    # 模型参数，可调
    model = lgb.LGBMRegressor(num_leaves=20,
                              max_depth=4,
                              learning_rate=0.08,
                              n_estimators=10000,
                              subsample=0.9,
                              feature_fraction=0.8,
                              reg_alpha=0.6,
                              reg_lambda=1.2,
                              random_state=42)
    oof = []
    prediction = test[['ID', 'QUEUE_ID']]
    prediction[target] = 0
    
    kfold = KFold(n_splits=5, random_state=42)
    for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(train, train[target])):
        
        X_train = train.iloc[trn_idx][feature_names]
        Y_train = train.iloc[trn_idx][target]
        X_val = train.iloc[val_idx][feature_names]
        Y_val = train.iloc[val_idx][target]
        
        lgb_model = model.fit(X_train, 
                              Y_train,
                              eval_names=['train', 'valid'],
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=0,
                              eval_metric='mse',
                              early_stopping_rounds=20)
        
        pred_val = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration_)
        df_oof = train.iloc[val_idx][[target, 'QUEUE_ID']].copy()
        df_oof['pred'] = pred_val
        oof.append(df_oof)
        
        pred_test = lgb_model.predict(test[feature_names], num_iteration=lgb_model.best_iteration_)
        prediction[target] += pred_test / kfold.n_splits
        
        del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
        gc.collect()
        
    df_oof = pd.concat(oof)
    mse_score = mean_squared_error(df_oof[target], df_oof['pred'])
    print('MSE:', mse_score)

    return prediction, mse_score

In [33]:
predictions = list()
scores = list()

for qid in tqdm(test.QUEUE_ID.unique()):    
    df = pd.DataFrame()
    for t in [f'cpu_{i}' for i in range(1,6)]:
        prediction, score = run_lgb_qid(train, test, target=t, qid=qid)
        if t == 'cpu_1':
            df = prediction.copy()
        else:
            df = pd.merge(df, prediction, on=['ID', 'QUEUE_ID'], how='left')            
        scores.append(score)

    predictions.append(df)

QUEUE_ID:297, target:cpu_1, train:21175, test:1142
MSE: 138.91911583774626
QUEUE_ID:297, target:cpu_2, train:21175, test:1142
MSE: 217.17200488097907
QUEUE_ID:297, target:cpu_3, train:21175, test:1142
MSE: 232.70612270018938
QUEUE_ID:297, target:cpu_4, train:21175, test:1142
MSE: 180.43915751558586
QUEUE_ID:297, target:cpu_5, train:21175, test:1142
MSE: 136.5363278388821
QUEUE_ID:85153, target:cpu_1, train:14344, test:390
MSE: 164.76878220650437
QUEUE_ID:85153, target:cpu_2, train:14344, test:390
MSE: 190.5119107829116
QUEUE_ID:85153, target:cpu_3, train:14344, test:390
MSE: 195.95275603053358
QUEUE_ID:85153, target:cpu_4, train:14344, test:390
MSE: 200.60419884333822
QUEUE_ID:85153, target:cpu_5, train:14344, test:390
MSE: 202.5039448059024
QUEUE_ID:291, target:cpu_1, train:8875, test:57
MSE: 82.52084312732273
QUEUE_ID:291, target:cpu_2, train:8875, test:57
MSE: 181.10624656374011
QUEUE_ID:291, target:cpu_3, train:8875, test:57
MSE: 237.7474596510213
QUEUE_ID:291, target:cpu_4, train:

MSE: 7.981065924653396
QUEUE_ID:298, target:cpu_4, train:20372, test:2
MSE: 8.172622105977954
QUEUE_ID:298, target:cpu_5, train:20372, test:2
MSE: 8.175060831559108



In [34]:
print('mean MSE score: ', np.mean(scores))

mean MSE score:  55.293087557044764


In [35]:
sub = pd.concat(predictions)

sub = sub.sort_values(by='ID').reset_index(drop=True)
sub.drop(['QUEUE_ID'], axis=1, inplace=True)
sub.columns = ['ID'] + [f'CPU_USAGE_{i}' for i in range(1,6)]

# 全置 0 都比训练出来的结果好
for col in [f'LAUNCHING_JOB_NUMS_{i}' for i in range(1,6)]:
    sub[col] = 0
    
sub = sub[['ID',
           'CPU_USAGE_1', 'LAUNCHING_JOB_NUMS_1', 
           'CPU_USAGE_2', 'LAUNCHING_JOB_NUMS_2', 
           'CPU_USAGE_3', 'LAUNCHING_JOB_NUMS_3', 
           'CPU_USAGE_4', 'LAUNCHING_JOB_NUMS_4', 
           'CPU_USAGE_5', 'LAUNCHING_JOB_NUMS_5']]

print(sub.shape)
sub.head()

(2996, 11)


,ID,CPU_USAGE_1,LAUNCHING_JOB_NUMS_1,CPU_USAGE_2,LAUNCHING_JOB_NUMS_2,CPU_USAGE_3,LAUNCHING_JOB_NUMS_3,CPU_USAGE_4,LAUNCHING_JOB_NUMS_4,CPU_USAGE_5,LAUNCHING_JOB_NUMS_5
0,1,89.124057,0,85.726408,0,87.722501,0,89.822449,0,89.903595,0
1,2,31.782745,0,30.500661,0,38.621049,0,42.610640,0,29.439808,0
2,3,38.176173,0,55.271222,0,15.833593,0,11.817200,0,4.577775,0
3,4,26.630747,0,14.064676,0,7.398893,0,5.425343,0,5.069402,0
4,5,2.932833,0,6.874329,0,11.777893,0,12.781514,0,10.295482,0


In [36]:
# 注意: 提交要求预测结果需为非负整数, 包括 ID 也需要是整数

sub['ID'] = sub['ID'].astype(int)

for col in [i for i in sub.columns if i != 'ID']:
    sub[col] = sub[col].apply(np.floor)
    sub[col] = sub[col].apply(lambda x: 0 if x<0 else x)
    sub[col] = sub[col].astype(int)
    
sub.head(10)

,ID,CPU_USAGE_1,LAUNCHING_JOB_NUMS_1,CPU_USAGE_2,LAUNCHING_JOB_NUMS_2,CPU_USAGE_3,LAUNCHING_JOB_NUMS_3,CPU_USAGE_4,LAUNCHING_JOB_NUMS_4,CPU_USAGE_5,LAUNCHING_JOB_NUMS_5
0,1,89,0,85,0,87,0,89,0,89,0
1,2,31,0,30,0,38,0,42,0,29,0
2,3,38,0,55,0,15,0,11,0,4,0
3,4,26,0,14,0,7,0,5,0,5,0
4,5,2,0,6,0,11,0,12,0,10,0
5,6,13,0,14,0,13,0,11,0,11,0
6,7,11,0,10,0,6,0,6,0,5,0
7,8,0,0,0,0,0,0,0,0,0,0
8,9,3,0,4,0,3,0,3,0,3,0
9,10,16,0,11,0,12,0,10,0,14,0


In [37]:
sub.to_csv('baseline_1113.csv', index=False)